In [199]:
import anndata as ad
import pandas as pd
import numpy as np
import altair as alt
from scipy.spatial.distance import cdist
import random

# Set to None to display all rows
pd.set_option('display.max_rows', 100)

## read and process data

In [3]:
adata = ad.read_h5ad("Z147_1_750.h5ad")

In [98]:
print(adata.obs_keys)
print(adata.obs['phenotype'].value_counts())


<bound method AnnData.obs_keys of AnnData object with n_obs × n_vars = 1110585 × 30
    obs: 'X_centroid', 'Y_centroid', 'column_centroid', 'row_centroid', 'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'imageid', 'phenotype', 'kmeans', 'kmeans_renamed', 'pickseq_roi', 'pickseq_roi_minimal', 'dermis_roi', 'epidermis_roi', 'general_roi', 'Kmeans_r', 'der_epider_roi', 'phenotype_proliferation', 'phenograph', 'spatial_expression_phenograph', 'spatial_expression_kmeans', 'phenograph_raw', 'phenograph_raw_minimal', 'phenotype_1', 'phenotype_1_tumor_kmeans', 'phenotype_1_tumor_kmeans_renamed', 'phenotype_final', 'spatial_expression_consolidated', 'MC', 'spatial_count_kmeans', 'phenotype_2', 'phenotype_2_tumor_kmeans', 'phenotype_2_tumor_kmeans_renamed', 'phenotype_large_cohort', 'phenotype_large_cohort_RJP', 'spatial_lda_kmeans', 'phenotype_2_tumor_kmeans_old', 'tumor_expression_trimmed', 'tumor_expression_TC', 'IM_roi', 'TC_ROI_IM_large', 

#### **First three observations look like:**

In [36]:
print(adata.obs.head(3))

                   X_centroid  Y_centroid  column_centroid  row_centroid  \
unmicst-1_750_1  36345.943182   16.662879     36345.943182     16.662879   
unmicst-1_750_2  36494.212121   46.916667     36494.212121     46.916667   
unmicst-1_750_3  36485.014706   52.926471     36485.014706     52.926471   

                 Area  MajorAxisLength  MinorAxisLength  Eccentricity  \
unmicst-1_750_1   792        40.112476        25.480990      0.772316   
unmicst-1_750_2   132        14.340782        12.703926      0.463953   
unmicst-1_750_3    68        10.909423         9.252049      0.529870   

                 Solidity    Extent  Orientation        imageid phenotype  \
unmicst-1_750_1  0.938389  0.673469     1.465017  unmicst-1_750   Unknown   
unmicst-1_750_2  0.929577  0.628571     0.812762  unmicst-1_750   Unknown   
unmicst-1_750_3  0.829268  0.472222    -0.257120  unmicst-1_750   Unknown   

                  kmeans kmeans_renamed pickseq_roi pickseq_roi_minimal  \
unmicst-1_750_1  U

#### **Data Shape:**

In [115]:
adata[list(adata.obs_names),:].X.shape

(1110585, 30)

### Sampling Schemes

Two possible sampling schemes:
1. Sample without replacement until each phenotype has some number, s, of samples.
2. Randomly draw some number, n, of samples (Altair caps at 5000 so we must use this)

In [50]:
# Method 1

# Identify all unique phenotypes
phenotypes = adata.obs["phenotype"].unique()

# Set up dictionary to store sampled indices
sampled_indices = {phenotype: [] for phenotype in phenotypes}

# List of indices to sample from
all_indices = list(range(adata.n_obs))

# Sampling until conditions are met
while True:
    # Randomly draw an index
    idx = random.choice(all_indices)
    
    # Determine the phenotype for this index
    phenotype = adata.obs.iloc[idx]["phenotype"]
    
    # Add the index to the appropriate phenotype group
    sampled_indices[phenotype].append(idx)
    
    # Check if all phenotypes have at least 30 samples
    if all(len(indices) >= 30 for indices in sampled_indices.values()):
        break

# Gather the final sampled indices
final_sampled_indices = [index for indices in sampled_indices.values() for index in indices]

# Subset the AnnData object
sampled_adata = adata[final_sampled_indices]


In [36]:
print(sampled_adata.obs['phenotype'].value_counts())

phenotype
Tumor                           31577
Unknown                         10924
Myeloid Lineage                  8285
Myofibroblast                    4065
Blood Vessels                    3613
APCs                             2415
T cells                          1828
Macrophages                      1785
CD11C+ PDL1+ cells                925
Keratinocytes                     863
Melanocytes                       507
Terminally Exhausted T cells      493
Patially Exhausted T cells        426
Regulatory T cells                305
Mast cells                        245
Cytotoxic T cells                  69
Langerhan cells                    30
Name: count, dtype: int64


In [317]:
# Method 2

# Define the number of samples you want
n_samples = 5000

# Number of observations (cells) in the dataset
n_obs = adata.n_obs

# Ensure the sample size does not exceed the number of available observations
if n_samples > n_obs:
    raise ValueError("Number of samples requested exceeds the number of available observations.")

# Generate random indices for sampling
random_indices = np.random.choice(n_obs, n_samples, replace=False)

# Subset the AnnData object using the random indices
sampled_adata = adata[random_indices, :]

# Print the shape to confirm the subset process
print(sampled_adata.shape)

(5000, 30)


### **Plot phenotype grid**

In [353]:
import altair as alt

# Assuming `adata` is your AnnData object
# adata = ad.read_h5ad('your_data.h5ad')

# Preparing the data
df = sampled_adata.obs[['X_centroid', 'Y_centroid', 'phenotype']]  # Replace 'phenotype' with the actual column name

# Convert the relevant data to a DataFrame
coordinates = df.reset_index()
#chart_radio = alt.binding_radio(options = list_options)
radio_select = alt.selection_point(
    fields=['phenotype'],
    bind=alt.binding_radio(
        options=df['phenotype'].unique().tolist(),
        name='Select Phenotype: '
    )
)

# Create the scatter plot with Altair
chart = alt.Chart(coordinates).mark_circle(size=60, opacity=0.5).encode(
    x='X_centroid:Q',
    y='Y_centroid:Q',
    color='phenotype:N',  # Nominal encoding for categorical data
    tooltip=['X_centroid', 'Y_centroid', 'phenotype']
).properties(
    width=800,
    height=600,
    title='Cell Centroid Coordinates by Phenotype'
).add_params(radio_select).transform_filter(radio_select)

# Display the chart
chart.save("XY_scatter_V1.html")  

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


### **Plot Phenotype Neighborhoods**

In [372]:
# Function to calculate normalized average distances within each phenotype
def calculate_normalized_distances(df,q):
    results = pd.DataFrame(index=df.index, columns=['normalized_avg_distance'])
    
    # Iterate over each unique phenotype
    for phenotype in df['phenotype'].unique():
        phenotype_data = df[df['phenotype'] == phenotype]
        if len(phenotype_data) > 1:
            coords = phenotype_data[['X_centroid', 'Y_centroid']].values
            distances = cdist(coords, coords, 'euclidean')  # Calculate pairwise distances

            quantile_thresholds = np.quantile(distances, q, axis=0)
            mask = (distances <= quantile_thresholds[:, np.newaxis]).astype(int)
            neighbor_distances = np.multiply(mask,distances)
            # Average distance per cell, excluding zero distance to self
            avg_distances = (neighbor_distances.sum(axis=1) - np.diag(distances)) / (len(phenotype_data) - 1)
            
            # Calculate phenotype-wide average distance
            phenotype_avg_distance = avg_distances.mean()
            # Normalize individual average distances by phenotype-wide average, then take inverse
            normalized_distances = np.power(avg_distances / phenotype_avg_distance,-1)
            results.loc[phenotype_data.index, 'normalized_avg_distance'] = normalized_distances

        else:
            results.loc[phenotype_data.index, 'normalized_avg_distance'] = 1
    return results

# Calculate normalized average distances
normalized_distances = calculate_normalized_distances(df,0.1)

# Add result to the original DataFrame
df['normalized_avg_distance'] = normalized_distances['normalized_avg_distance']
print(df.head())

                        X_centroid    Y_centroid      phenotype  \
unmicst-1_750_87105   10610.500000   3399.000000  Myofibroblast   
unmicst-1_750_310496   9143.465517   9977.591954          Tumor   
unmicst-1_750_468348   8906.988827  12215.469274          Tumor   
unmicst-1_750_400074  27764.987952  11298.927711          Tumor   
unmicst-1_750_415111  11935.690647  11495.949640          Tumor   

                     normalized_avg_distance  
unmicst-1_750_87105                 0.682964  
unmicst-1_750_310496                1.220351  
unmicst-1_750_468348                1.169139  
unmicst-1_750_400074                0.569277  
unmicst-1_750_415111                1.263545  


/var/folders/zf/q6vckqdx64v8bb5_y2dtjvjw0000gn/T/ipykernel_73037/1125307436.py:21: RuntimeWarning: invalid value encountered in divide
  normalized_distances = np.power(avg_distances / phenotype_avg_distance,-1)
/var/folders/zf/q6vckqdx64v8bb5_y2dtjvjw0000gn/T/ipykernel_73037/1125307436.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['normalized_avg_distance'] = normalized_distances['normalized_avg_distance']


In [374]:
radio_select = alt.selection_point(
    fields=['phenotype'],
    bind=alt.binding_radio(
        options=df['phenotype'].unique().tolist(),
        name='Select Phenotype: '
    )
)

# Create the scatter plot with Altair, using the normalized_avg_distance for color
chart = alt.Chart(df).mark_circle(size=60, opacity=0.7).encode(
    x='X_centroid:Q',
    y='Y_centroid:Q',
    color=alt.Color('normalized_avg_distance:Q', scale=alt.Scale(scheme='reds'), title='Concentration'),
    tooltip=['X_centroid', 'Y_centroid', 'phenotype', 'normalized_avg_distance']
).properties(
    width=800,
    height=600,
    title='Cell Centroid Coordinates with Concentration Encoding'
).add_params(radio_select).transform_filter(radio_select)

# Display the chart
chart.save("XY_concentration_V2.html") 

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
